# 2 Layer search agent
- search for local documnet when it present
- search for the internet when no information

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
import sys
sys.path.append("../../")
from src.config import Configuration
from langchain.tools.retriever import create_retriever_tool
conf = Configuration()

In [4]:
conf.set_tavily_token()
tavily = TavilySearchResults()

## Server RAG

In [5]:
from src.rag.hybrid_gemini import HybridGeminiRag
INDEX = "labse-major"
MODEL = "sentence-transformers/LaBSE"
rag = HybridGeminiRag(config=conf, es_index=INDEX, embed_model=MODEL)


From (44, 4) to Counter({2: 37, 1: 6, 3: 1})
From (44, 4) to 44


In [6]:
rag_tool = create_retriever_tool(
    retriever=rag.ensemble_retriever,
    name="rag_hybrid_search",
    description="Find information about the training programs (majors) of Ton Duc Thang University. You must use this tool to answer questions related to the training programs (majors) of Ton Duc Thang University.",
)

## Local RAG

In [7]:
from src.prepare.data_load import DocDataLoader

doc_loader = DocDataLoader()
docs = doc_loader.load_major_docs_full()
d1 = docs[15]
d1.page_content[:30]

From (44, 4) to 44


'Kỹ thuật hoá học là ngành học'

In [8]:
from src.chain.followup import LocalFollowUpChain
followup = LocalFollowUpChain(config=conf, doc_content=d1.page_content)

In [21]:
q = "Xin bạn hãy viết một mô tả chính về nội dung văn bản được cho theo dạng mô tả guideline bằng một câu"
q = "văn bản được cho nói về nội dung chính là gì?"
a = followup.answer(question=q)
a

'Nội dung chính của văn bản là giới thiệu về ngành Kỹ thuật hóa học tại trường Đại học Tôn Đức Thắng, bao gồm chương trình đào tạo, cơ hội nghề nghiệp và các hoạt động ngoại khóa dành cho sinh viên.'

In [22]:
a = "ngành Kỹ thuật hóa họ"

In [23]:
def local_retrieve(question:str) -> str:
    
    return d1.page_content

desc = "Find information about the {main} of Ton Duc Thang University. You must use this tool to answer questions related to {main} of Ton Duc Thang University."

local_tool = create_retriever_tool(
    retriever=local_retrieve,
    name="local_hybrid_search",
    description=desc.format(main = a),
)

## Agent

In [24]:
from langchain import agents
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# Choose the LLM that will drive the agent
llm = conf.get_gemini_pro(True)

tools = [local_tool, rag_tool]
# Construct the JSON agent
agent = create_openai_functions_agent(llm, tools, prompt)

In [25]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [26]:
from langsmith.run_helpers import traceable

conf.enable_tracing(project="LEARN")

@traceable(tags=["agent"])
def run_agent(q:str):
    a = agent_executor.invoke({"input": q})
    return a

In [27]:
q = "em muốn biết thêm thông tin về ngành kỹ thuật hóa học của trường ạ"
a = run_agent(q)
a



> Entering new AgentExecutor chain...
**Ngành Kỹ thuật Hóa học tại Trường**

**Giới thiệu**

Ngành Kỹ thuật Hóa học tại trường chúng tôi là một chương trình học tập toàn diện cung cấp cho sinh viên nền tảng vững chắc về các nguyên tắc khoa học và kỹ thuật liên quan đến quá trình thiết kế, vận hành và tối ưu hóa các hệ thống hóa học.

**Mục tiêu chương trình**

Chương trình Kỹ thuật Hóa học của chúng tôi nhằm mục đích:

* Trang bị cho sinh viên kiến thức và kỹ năng cần thiết để giải quyết các vấn đề phức tạp trong ngành công nghiệp hóa học.
* Phát triển khả năng phân tích, giải quyết vấn đề và thiết kế của sinh viên.
* Chuẩn bị sinh viên cho sự nghiệp thành công trong các lĩnh vực liên quan đến hóa học.

**Điểm nổi bật của chương trình**

* **Giáo trình toàn diện:** Chương trình học bao gồm các khóa học về hóa học, toán học, vật lý, kỹ thuật và các lĩnh vực liên quan.
* **Cơ sở vật chất hiện đại:** Sinh viên có quyền truy cập vào các phòng thí nghiệm và cơ sở nghiên cứu tiên tiến.
* *

{'input': 'em muốn biết thêm thông tin về ngành kỹ thuật hóa học của trường ạ',
 'output': '**Ngành Kỹ thuật Hóa học tại Trường**\n\n**Giới thiệu**\n\nNgành Kỹ thuật Hóa học tại trường chúng tôi là một chương trình học tập toàn diện cung cấp cho sinh viên nền tảng vững chắc về các nguyên tắc khoa học và kỹ thuật liên quan đến quá trình thiết kế, vận hành và tối ưu hóa các hệ thống hóa học.\n\n**Mục tiêu chương trình**\n\nChương trình Kỹ thuật Hóa học của chúng tôi nhằm mục đích:\n\n* Trang bị cho sinh viên kiến thức và kỹ năng cần thiết để giải quyết các vấn đề phức tạp trong ngành công nghiệp hóa học.\n* Phát triển khả năng phân tích, giải quyết vấn đề và thiết kế của sinh viên.\n* Chuẩn bị sinh viên cho sự nghiệp thành công trong các lĩnh vực liên quan đến hóa học.\n\n**Điểm nổi bật của chương trình**\n\n* **Giáo trình toàn diện:** Chương trình học bao gồm các khóa học về hóa học, toán học, vật lý, kỹ thuật và các lĩnh vực liên quan.\n* **Cơ sở vật chất hiện đại:** Sinh viên có quyền